In [1]:
import carla
import random
import time
import math

import numpy as np
import cv2

In [2]:
# Connect to the CARLA Simulator
client = carla.Client('localhost', 2000)
world = client.get_world()

In [3]:
bp_lib = world.get_blueprint_library()
spawn_points = world.get_map().get_spawn_points()

In [4]:
vehicle_bp = bp_lib.find("vehicle.lincoln.mkz_2020")
vehicle = world.try_spawn_actor(vehicle_bp, random.choice(spawn_points))

spectator = world.get_spectator()
transform = carla.Transform(vehicle.get_transform().transform(carla.Location(x=-4, z=2.5)), vehicle.get_transform().rotation)
spectator.set_transform(transform)

In [ ]:
# camera_bp = bp_lib.find("sensor.camera.rgb")
# camera_bp.set_attribute("sensor_tick", "0.2")

# camera_init_trans = carla.Transform(carla.Location(z=2, x=0.4))
# camera = world.spawn_actor(camera_bp, camera_init_trans, attach_to=vehicle)

# # time.sleep(0.2)
# spectator.set_transform(camera.get_transform())
# # camera.destroy()

# camera.listen(lambda image: image.save_to_disk('output/%06d.png' % image.frame))

In [ ]:
# camera.stop()

In [5]:
# Create a camera sensor blueprint
# camera_bp = bp_lib.find("sensor.camera.rgb")
camera_bp = bp_lib.find("sensor.camera.depth")


# Set camera attributes
camera_bp.set_attribute("image_size_x", "120")  # Reduce width
camera_bp.set_attribute("image_size_y", "90")  # Reduce height
camera_bp.set_attribute("sensor_tick", "0.10")   # Capture one frame every 0.5 seconds


# Initialize the camera transformation (relative to the vehicle)
camera_init_trans = carla.Transform(carla.Location(z=1.6, x=0.4))

# Spawn the camera sensor attached to the vehicle
camera = world.spawn_actor(camera_bp, 
                           camera_init_trans, 
                           attach_to=vehicle)

# Define a function to process and save images
# def process_image(image):
#     image.convert(carla.ColorConverter.Raw)  # Optional: Adjust color format
#     image.save_to_disk("output/%06d.png" % image.frame)

# Attach a listener to the camera
# camera.listen(process_image)


# Run for a few seconds to collect data (then destroy the camera)
# time.sleep(10)  # Collect images for 5 seconds
# camera.destroy()


In [6]:
def camera_callback(image, data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data), 
                                    (image.height, image.width, 4))

In [7]:
image_w = camera_bp.get_attribute("image_size_x").as_int()
image_h = camera_bp.get_attribute("image_size_y").as_int()


camera_data = {'image': np.zeros((image_h, image_w, 4), dtype=np.uint8)}

camera.listen(lambda image: camera_callback(image, camera_data))

In [8]:
vehicle.set_autopilot()

In [ ]:
cv2.namedWindow("RGB Camera", cv2.WINDOW_AUTOSIZE)
cv2.imshow("RGB Camera", camera_data['image'])
cv2.waitKey(1)


while True:
    cv2.imshow("RGB Camera", camera_data['image'])
    
    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()